In [1]:
from wdtools import *

C:\Users\clid1852\AppData\Anaconda3\envs\geoenv\lib\site-packages\geopandas\_compat.py:106: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.
  warnings.warn(


In [2]:
set_ID = 'Set001'

In [3]:
# this will take about 17 minutes
start = time.time()
all_taxlot = combine_taxlot()
end = time.time()
print(f'it took {round((end - start)/60, 0)} minutes to complete')

it took 16.0 minutes to complete


In [4]:
# this will take about 1 minute
start = time.time()
res = check_corrected_data(setID=set_ID, all_taxlot=all_taxlot, export=False)
end = time.time()
print(f'it took {round((end - start)/60, 0)} minutes to complete')

it took 1.0 minutes to complete


In [5]:
ngdf = res[0]

In [6]:
wd_df = combine_wd_tables(set_ID)

In [7]:
wd_df.shape

(1420, 25)

In [8]:
len(ngdf.record_ID.unique())

1331

In [9]:
wd_df.columns

Index(['wetdet_delin_number', 'trsqq', 'parcel_id', 'address_location_desc',
       'city', 'county', 'site_name', 'site_desc', 'latitude', 'longitude',
       'Coord-Source', 'DocumentName', 'DecisionLink', 'is_batch_file',
       'status_name', 'received_date', 'response_date',
       'reissuance_response_date', 'project_id', 'site_id', 'record_ID',
       'notes', 'recyear', 'IDyear', 'missinglot'],
      dtype='object')

In [10]:
setgdf = gpd.read_file(os.path.join(inpath, 'GIS', 'Join_Statewide.gdb'), layer=f'WD_{set_ID}_Combined')

In [11]:
df_wlots_to_check = res[3]

In [12]:
df_wlots_to_check.columns

Index(['trsqq', 'parcel_id', 'Coord-Source', 'DocumentName', 'DecisionLink',
       'is_batch_file', 'status_name', 'received_date', 'response_date',
       'reissuance_response_date', 'project_id', 'site_id', 'record_ID',
       'notes', 'recyear', 'IDyear', 'missinglot', 'wetdet_delin_number'],
      dtype='object')

In [13]:
setgdf.columns

Index(['Record_ID', 'wetdet_delin_number', 'trsqq', 'parcel_id',
       'address_location_desc', 'city', 'county', 'site_name', 'site_desc',
       'latitude', 'longitude', 'Coord_Source', 'DocumentName', 'DecisionLink',
       'is_batch_file', 'status_name', 'received_date', 'response_date',
       'reissuance_response_date', 'project_id', 'site_id', 'lot_num', 'lot',
       'Match_Year', 'ORTAXLOT', 'Match_found', 'Manual_note', 'Shape_Length',
       'Shape_Area', 'Edits_Complete', 'matcher2', 'geometry'],
      dtype='object')

In [14]:
# missed records in the existing output
wd_df.shape[0] - len(setgdf.Record_ID.unique())

58

In [15]:
missed_match_ID = compare_data_report(gdf=ngdf, setID=set_ID)[0]

C:\Users\clid1852\AppData\Anaconda3\envs\geoenv\lib\site-packages\geopandas\geodataframe.py:1299: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super(GeoDataFrame, self).__getitem__(key)


In [16]:
# missed records in the matched
len(missed_match_ID)

23

In [17]:
setgdf_s = setgdf[setgdf.Record_ID.isin(df_wlots_to_check.record_ID)]

In [18]:
setgdf_s.loc[:, 'record_ID'] = setgdf_s.loc[:, 'Record_ID'].astype(int)

C:\Users\clid1852\AppData\Anaconda3\envs\geoenv\lib\site-packages\geopandas\geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


In [19]:
rIDs = setgdf_s.Record_ID.unique()

In [20]:
df_wlots_to_check[df_wlots_to_check.record_ID.isin(rIDs)][['record_ID', 'wetdet_delin_number', 'trsqq', 'parcel_id']].merge(setgdf_s[['record_ID', 'wetdet_delin_number', 'trsqq', 'parcel_id']], on='record_ID')

,record_ID,wetdet_delin_number_x,trsqq_x,parcel_id_x,wetdet_delin_number_y,trsqq_y,parcel_id_y
0,83,WD2020-0185,05N10W014,501,WD2020-0185,05N10W14,501
1,176,WD2022-0187,06N10W22,8100,WD2022-0187,06N10W22AB,8100
2,182,WD2017-0016,28S15W36AD,1302,WD2017-0016,28S15W36AD,1302
3,223,WD2019-0122,26S14W02D,300,WD2019-0122,26S14W02D,300
4,235,WD2019-0338,24S13W34C,140p,WD2019-0338,24S13W34C,140p
5,237,WD2019-0338,24S13W00,200p,WD2019-0338,24S13W00,200p
6,416,WD2020-0067,17S12E10C,"400,500 all partial",WD2020-0067,17S12E10C,"400,500 all partial"
7,477,WD2020-0586,20S10E25BB,400p,WD2020-0586,20S10E25B,400p
8,484,WD2020-0668,18S12E08DA,"ROW, 6317,6303,3604",WD2020-0668,18S12E08DA,"ROW, 6317,6303,3604"
9,494,WD2021-0066,18S12W25B,400,WD2021-0066,18S12E25B,400


In [21]:
all_taxlot.columns

Index(['year', 'ORTaxlot', 'geometry'], dtype='object')

In [22]:
len(ngdf.ORTaxlot.unique())

1989

In [61]:
df_wlots_to_check.shape

(28, 20)

In [131]:
start = time.time()
tocheck_df = reorganize_tocheck(df_wlots_to_check)
end = time.time()
print(f'it took {round((end - start)/60, 0)} minutes to complete')

it took 6.0 minutes to complete


In [132]:
tocheck_df.head(3)

,trsqq,parcel_id,Coord-Source,DocumentName,DecisionLink,is_batch_file,status_name,received_date,response_date,reissuance_response_date,project_id,site_id,record_ID,notes,recyear,IDyear,missinglot,wetdet_delin_number,trsqq_n,n_trsqq
28,08N10W21B,"3400,2900,2800,SW 2nd street ROW (all partial)",DSL Verified,WD2018-0042final,https://docs.dsl.state.or.us/PublicReview/docv...,N,Approved,2018-01-31,2018-03-06,NaN,NaN,139396,29,ROW & partial,2018,2018,N,WD2018-0042,"[08N10W21AA, 08N10W21AB, 08N10W21AC, 08N10W21A...",17
82,05N10W014,501,DSL Verified,WD20200185 AgencyDecision (3),https://docs.dsl.state.or.us/PublicReview/Elec...,Y,Completed by DSL,2020-03-31,2020-04-22,NaN,79726.0,132445,83,None,2020,2020,N,WD2020-0185,"[05N10W1400, 05N10W14CA, 05N10W14DB, 05N10W14DC]",4
175,06N10W22,8100,DSL Verified,NaN,NaN,N,Information Requested,2022-04-01,NaT,NaN,82664.0,138954,176,None,2022,2022,N,WD2022-0187,"[06N10W2200, 06N10W22AB, 06N10W22AC, 06N10W22B...",12


In [133]:
tocheck_df.n_trsqq.unique()

array([17,  4, 12, 16,  2,  5,  7, 13, 20,  1], dtype=int64)

In [134]:
len(tocheck_df[tocheck_df.n_trsqq == 1].trsqq.unique())

6

In [109]:
trsqq_to_check = '24S32E16'

In [115]:
closematch = difflib.get_close_matches(trsqq_to_check, trsqq)
trsqq_matched = unique(closematch)
checktaxlot = [*map(trsqq_dict.get, trsqq_matched)]

In [116]:
checktaxlot

['1324.00S33.00E1600--000000900']

In [118]:
string_to_search = trsqq_matched[0][0:8]
search_res = unique([i for i in unique(df.trsqq) if string_to_search in i])

In [127]:
type(search_res)

list

In [129]:
type([i for i in trsqq_dict if trsqq_dict[i]==checktaxlot[0]])

list

In [121]:
get_maybe_taxlot(trsqq_to_check)

set()

In [65]:
res = get_trsqq_list()
trsqq = res[0] 
trsqq_dict = res[1]
df = res[2]

In [84]:
closematch = difflib.get_close_matches(trsqq_to_check, trsqq)

In [93]:
trsqq_matched = unique(closematch)

In [88]:
checktaxlot = [*map(trsqq_dict.get, trsqq_matched)]

In [89]:
checktaxlot

['0405.00N10.00W14DC--0000ROADS']

In [99]:
string_to_search = trsqq_matched[0][0:8]

In [100]:
string_to_search

'05N10W14'

In [101]:
search_res = unique([i for i in unique(df.trsqq) if string_to_search in i])

In [102]:
search_res

['05N10W1400', '05N10W14CA', '05N10W14DB', '05N10W14DC']

In [72]:
len(search_res)

17

In [74]:
taxlots = unique([*map(trsqq_dict.get, search_res)])

In [75]:
values = df.loc[df.ORTaxlot.isin(taxlots), 'trsqq'].values